# Assignment sarsa NN

In [1]:
### Import 

import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *


size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [2]:
## INITIALISE THE ENVIRONMENT

env=Chess_Env(size_board)

In [3]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S,X,allowed_a=env.Initialise_game()                       # INTIALISE GAME

print(S)                                                  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ',env.check)                                # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))    # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO


for i in range(5):
    
    a,_=np.where(allowed_a==1)                  # FIND WHAT THE ALLOWED ACTIONS ARE
    a_agent=np.random.permutation(a)[0]         # MAKE A RANDOM ACTION

    S,X,allowed_a,R,Done=env.OneStep(a_agent)   # UPDATE THE ENVIRONMENT
    
    
    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print(S)
    print(R,'', Done)
    print('check? ',env.check)
    print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))
    
    
    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break
        


[[0 0 3 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 2 1 0]]
check?  0
dofk2  2

[[0 0 0 3]
 [0 0 0 0]
 [0 0 1 0]
 [0 2 0 0]]
0  0
check?  0
dofk2  1

[[0 0 3 0]
 [0 0 0 0]
 [0 2 1 0]
 [0 0 0 0]]
0  0
check?  0
dofk2  0

[[0 0 0 3]
 [0 2 0 0]
 [0 0 1 0]
 [0 0 0 0]]
0  0
check?  0
dofk2  0

[[0 0 3 0]
 [2 0 0 0]
 [0 0 1 0]
 [0 0 0 0]]
0  0
check?  0
dofk2  1

[[0 0 0 3]
 [2 0 0 0]
 [0 0 0 1]
 [0 0 0 0]]
0  0
check?  0
dofk2  1


In [4]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S,X,allowed_a=env.Initialise_game()
N_episodes=1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):
    
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        
        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 
        
        a,_=np.where(allowed_a==1)

        
        a_agent=np.random.permutation(a)[0]


        S,X,allowed_a,R,Done=env.OneStep(a_agent)
        
        
        if Done:
            
            
            R_save_random[n]=np.copy(R)
            N_moves_save_random[n]=np.copy(i)

            break

        i=i+1                               # UPDATE THE COUNTER



# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW 
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:',np.mean(R_save_random),'Number of steps: ',np.mean(N_moves_save_random))



Random_Agent, Average reward: 0.188 Number of steps:  7.191


In [5]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 


S,X,allowed_a=env.Initialise_game()
N_a=np.shape(allowed_a)[0]# TOTAL NUMBER OF POSSIBLE ACTIONS


N_in=np.shape(X)[0]    ## INPUT SIZE
N_h=200                ## NUMBER OF HIDDEN NODES


## INITALISE YOUR NEURAL NETWORK...
W1=np.random.uniform(0,0,[N_in,N_h])/(N_in+N_h)
b1=np.zeros([N_h])

W2=np.random.uniform(0,0,[N_h,N_a])/(N_in+N_h)
b2=np.zeros([N_a])


# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.05    # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY 
##Original value of epsilon: 0.2
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE

#Original N_episodes: 100'000
N_episodes = 10000 # THE NUMBER OF GAMES TO BE PLAYED 

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])
N_moves_save = np.zeros([N_episodes, 1])


##Added Eligibility Trace; parameters taken from lab 3
eligibility_trace=True
if eligibility_trace:
    lamb=0.3
    eta=0.08


In [6]:
##Keeping track of the index to be able to get the right action
def epsilongreedy(Qval_allowed, idx_allowed, epsilon):

    N_a=np.shape(Qval_allowed)[0]
    rand_value=np.random.uniform(0,1)
    rand_a=rand_value<epsilon

    if rand_a==True:
        a=np.random.choice(idx_allowed)

    else:
        best=np.argmax(Qval_allowed)
        a=idx_allowed[best]
       
    return a

In [7]:
def sig(x):
    return 1 / (1 + np.exp(-x))  

def Forwardprop(x, W1, b1, W2, b2):
    hid_layer = np.dot(x, W1)+b1
    hid_layer_act = sig(hid_layer)
    out_layer=np.dot(hid_layer_act, W2)+b2
    out_layer_act =sig(out_layer)
    
    return out_layer_act, hid_layer_act


##done similarly to the "chess student" file
#W2 : only modifying the weights going to a_agent
#W1 : as the features are represented with all the 58 entries (unlike the action taken), all weights are updated (?)
def Backpropagation(delta, out_layer_act, hid_layer_act, x,  W1, W2, b1, b2):
    
    delta_W2=eta*delta*(hid_layer_act * (1-hid_layer_act))
    delta_W1=eta*np.outer(x, (delta*W2[:, a_agent]*(hid_layer_act *(1-hid_layer_act))).T)

    W2[:,a_agent]=W2[:,a_agent]+eta*delta_W2
    b2[a_agent]=b2[a_agent]+eta*delta
    
    W1=W1+eta*delta_W1
    b1=b1+eta*delta*(W2[:,a_agent]).dot((hid_layer_act *(1-hid_layer_act)))                  

    return  W1,  W2[:, a_agent], b1, b2[a_agent]

In [8]:
# TRAINING LOOP BONE STRUCTURE...

for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS
    
    S,X,allowed_a=env.Initialise_game()      ## INITIALISE GAME
    
    #Forward pass neural network
    Q_values, hid_layer_act = Forwardprop(X, W1, b1, W2, b2)
     
    #Get the index of the aLlowed actions
    idx_allowed,_=np.where(allowed_a==1)
            
    #Selecting the action with e-greedy
    Q_values_allowed=Q_values[idx_allowed]
    a_agent = epsilongreedy(Q_values_allowed, idx_allowed, epsilon_f)
    
    # Initialise eligibility traces 
    #e1: everything gets updated 
    #e2: only the action taken gets updated
    if eligibility_trace==True: 
        e1=np.zeros([N_in,N_h])
        e2=np.zeros([N_h, N_a])

    
    while Done==0:                           ## START THE EPISODE

        if eligibility_trace:
            e1=e1+1
            e2[:, a_agent]=e2[:, a_agent]+1

        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
   
        ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
        if Done==1:
            #calculate the error
            delta=R-Q_values[a_agent]
            #backpropagate the error
            W1, W2[:, a_agent], b1, b2[a_agent]= Backpropagation(delta, Q_values, hid_layer_act, X, W1, W2, b1, b2)
        
            if eligibility_trace:    
                W1=W1+eta*delta*e1 
                W2[:, a_agent]=W2[:, a_agent]+eta*delta*e2[:, a_agent]
        
            R_save[n]=np.copy(R)
            N_moves_save[n]=np.copy(i)
        
            ##TWO CHOICES: if R==1, then checkmate. Else, draw.

            break
        
        # IF THE EPISODE IS NOT OVER...
        else:
            #Get the qvalues of the next state
            Q_values_next, _ = Forwardprop(X_next, W1, b1, W2, b2)
            #selecting the qvalues of the allowed actions
            idx_allowed_next,_=np.where(allowed_a_next==1)
            Q_values_allowed_next=Q_values_next[idx_allowed_next]
            #select the action with e-greedy
            a_agent_next = epsilongreedy(Q_values_allowed_next, idx_allowed_next, epsilon_f)
            
            #Computing the error
            delta=R+gamma*Q_values_next[a_agent_next]-Q_values[a_agent]
            #backpropagate the error and update the weights
            W1, W2[:, a_agent], b1, b2[a_agent] = Backpropagation(delta, Q_values, hid_layer_act, X, W1, W2, b1, b2)
 
            if eligibility_trace:    
                W1=W1+eta*delta*e1  
                e1=gamma*lamb*e1
                W2[:, a_agent]=W2[:, a_agent]+eta*delta*e2[:, a_agent]  
                e2=gamma*lamb*e2
      
            
        # NEXT STATE AND CO. BECOME ACTUAL STATE...     
        S=np.copy(S_next)
        X=np.copy(X_next)
        a_agent = np.copy(a_agent_next)
        allowed_a = np.copy(allowed_a_next)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS

print('My Agent, Average reward:',np.mean(R_save),'Number of steps: ',np.mean(N_moves_save))
 

C:/Users/euxvp/AppData/Local/Temp/xpython_14332/1995278786.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


My Agent, Average reward: 0.53 Number of steps:  18.3561


In [ ]:
with ET (only e1, no e2)
My Agent, Average reward: 0.5142 Number of steps:  14.1718
My Agent, Average reward: 0.5056 Number of steps:  14.4996
for 10 000 episodes

without ET 
My Agent, Average reward: 0.3728 Number of steps:  22.0663     
        
with ET, e1 and e2
My Agent, Average reward: 0.5075 Number of steps:  13.8161
        
With weights initialized like in chess students and not 0 0 ; without ET
My Agent, Average reward: 0.4157 Number of steps:  19.1301
        
with weight " " and trace
My Agent, Average reward: 0.4802 Number of steps:  13.1398
        
with weights and trace but only e2[:, a_agent]
My Agent, Average reward: 0.5396 Number of steps:  14.9958
        
with weights and trace but only e2[:, a_agent]
My Agent, Average reward: 0.52478 Number of steps:  15.96016
but for 100 000 episodes

In [ ]:
##Plot
import matplotlib.pyplot as plt
import pandas as pd

R = pd.DataFrame(R_save)
Moves = pd.DataFrame(N_moves_save)

ema_r = R.ewm(com=500).mean()

ema_mo = Moves.ewm(com=500).mean()

fig, (rew, mov) = plt.subplots(2)

Time=np.array(range(len(ema_r)))

mov.scatter(Time, ema_mo, label="# moves", color="red")
rew.scatter(Time, ema_r, label="Reward", color="blue")

plt.xlabel("Episodes")

plt.legend()
plt.show()


